In [1]:
import pandas as pd
import numpy as np
import torch as T
from torch import nn
import torch.nn.functional as F

import chemprop
import rdkit

import pickle as pkl
import gzip

from tqdm.notebook import tqdm
import argparse

In [2]:
import covid
import covid.datasets
from covid.modules import *
from covid.data import *
from covid.model import *
from covid.schedulers import LinearWarmupScheduler

from covid.modules.chemistry import MPNEncoder

In [3]:
DROPOUT_RATE = 0.2
BATCH_SIZE = 16

In [4]:
chem_model = MPNEncoder(
    layers_per_message=3, 
    hidden_size=300,
    dropout=DROPOUT_RATE
)

In [20]:
chem_graphs.bond_fdim

147

In [18]:
chem_model.atom_messages

False

In [5]:
protein_model = create_protein_model(dropout=DROPOUT_RATE)

In [6]:
model = CovidModel(chem_model, protein_model, dropout=DROPOUT_RATE)

In [7]:
DEVICE = 'cpu'

In [8]:
model.to(DEVICE);

In [9]:
data = covid.datasets.StitchDataset()

In [10]:
dataloader = T.utils.data.DataLoader(
    data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn = covid.datasets.collate_stitch_data
)

In [11]:
optim = T.optim.Adam(model.parameters())
scheduler = LinearWarmupScheduler(optim, 2000)

In [12]:
chem_model(chem_graphs, chem_features)

NameError: name 'chem_graphs' is not defined

In [13]:
chemprop.models.mpn.get_bond_fdim(covid.utils.CHEMPROP_ARGS)

14

In [14]:
model(chem_graphs, chem_features, proteins)

NameError: name 'chem_graphs' is not defined

In [15]:
idx = 0

for batch in dataloader:
    chem_graphs, chem_features, proteins, targets = batch
    
    chem_graphs = chem_graphs.to(DEVICE)
    chem_features = chem_features.to(DEVICE)
    proteins = proteins.to(DEVICE)
    targets = targets.to(DEVICE)
    
    model.zero_grad()
    result = model(chem_graphs, chem_features, proteins)
    
    loss = F.binary_cross_entropy(result, target)
    loss.backward()
    
    optim.step()
    scheduler.step()
    idx += 1
    
    if idx >= 10:
        break
    

RuntimeError: size mismatch, m1: [901 x 147], m2: [14 x 300] at C:\w\1\s\tmp_conda_3.7_100118\conda\conda-bld\pytorch_1579082551706\work\aten\src\TH/generic/THTensorMath.cpp:136

In [77]:
next(iter(T.utils.data.DataLoader(
    data, batch_size=64, shuffle=True, collate_fn = collate_stitch_data
)))

(<covid.data.BatchMolGraph at 0x2323236c888>,
 tensor([[1.6959e+00, 1.2345e+03, 2.3413e+01,  ..., 0.0000e+00, 0.0000e+00,
          3.7968e-01],
         [1.6558e+00, 1.2393e+03, 2.9284e+01,  ..., 0.0000e+00, 0.0000e+00,
          1.1291e-01],
         [2.0560e+00, 9.9724e+02, 2.2198e+01,  ..., 0.0000e+00, 0.0000e+00,
          1.8802e-01],
         ...,
         [1.6273e+00, 1.3964e+03, 2.2336e+01,  ..., 0.0000e+00, 0.0000e+00,
          4.9215e-01],
         [1.4400e-06, 2.3728e+02, 9.3010e+00,  ..., 0.0000e+00, 0.0000e+00,
          4.5069e-01],
         [2.1133e+00, 5.7567e+02, 1.3786e+01,  ..., 1.0000e+00, 0.0000e+00,
          8.2997e-01]]),
 <ProteinBatch[64](torch.Size([1, 23, 43425])) offsets=(0, 294, 1213, 3098, 3446, 4541, 4728, 5943, 6572, 8035, 8575, 8760, 8987, 9158, 9724, 11067, 11188, 11876, 13322, 13805, 14475, 14953, 15800, 15904, 16207, 17077, 17473, 18244, 18855, 19948, 20719, 21959, 22594, 22937, 23660, 24349, 25137, 26501, 27724, 28234, 29237, 30498, 31642, 32141,